In [2]:
%load_ext autoreload
%autoreload 2
from EPECinterface import *
from gurobipy import *
import itertools
import csv

In [3]:
def printVars(vv, prefix = "", tol=1e-6):
    for vvv in vv:
        if abs(vv[vvv].X) >= tol:
            print(prefix,vvv,"\t---\t",vv[vvv].X)

# Sets

In [9]:
dirty = ["coal", "gas"]
green = ["wind", "solar"]

countries = ["c1", "c2"]
producers = ["p1_1", "p1_2", "p2_1", "p2_2"]

domesticity = tuplelist([("c1","p1_1"), ("c1","p1_2"),
               ("c2","p2_1"), ("c2","p2_2")
              ])

scenario = ["s"+str(i+1) for i in range(1
                                       )]

data = inputData(dirty, green, countries, producers, domesticity, scenario)

In [10]:
# mylist = readInputFromFile("inputs.csv")
# baseData = mylist[0]
data = inputData(dirty, green, countries, producers, domesticity, scenario)
data.buildFromList([0 for i in range(114)])
data.probability = {ss:1/len(data.scenario) for ss in data.scenario}

### Uneditables

In [11]:
# Capacity factor
for pp,ss in itertools.product(data.producers, data.scenario):
    data.CapacityFactor[pp,'wind',ss] = 0.348
    data.CapacityFactor[pp,'solar',ss] = 0.245

# Emission factor
data.Emission['coal'] = 0.76
data.Emission['gas'] = 0.36 
data.Emission['solar'] = 0 
data.Emission['wind'] = 0

# Uninteresting stuff
for pp in data.producers:
    data.InitCredits[pp] = 0
for vv in data.InitLeaderCredits:
    data.InitLeaderCredits[vv] = 0
for vv in data.InvestValue:
    data.InvestValue[vv] = 0
for vv in data.InvestCrossValue:
    data.InvestCrossValue[vv] = 0
for vv in data.EmissionValue:
    data.EmissionValue[vv] = 0
for vv in data.EmissionValueQuad:
    data.EmissionValueQuad[vv] = 0
for vv in data.EmissionCrossValue:
    data.EmissionCrossValue[vv] = 0

### Editables

In [36]:
# supplyIntercept and supplySlope
data.cciInt = 0
data.cciSlope = 0
# Demand intercept and slope
for cc,ss in itertools.product(data.countries, data.scenario):
    data.DemInt[cc,ss] = 200
    data.DemSlope[cc,ss] = 0.026

# Production costs
for pp in data.producers:
    data.LinProdCost[pp, 'coal'] = 42
    data.LinProdCost[pp, 'gas'] = 37
    data.LinProdCost[pp, 'solar'] = 0
    data.LinProdCost[pp, 'wind'] = 0
    #
    data.QuadProdCost[pp, 'coal'] = 0.0075
    data.QuadProdCost[pp, 'gas'] = 0.0095
    data.QuadProdCost[pp, 'solar'] = 0
    data.QuadProdCost[pp, 'wind'] = 0
# Investment cost
data.RPS = dict()
for pp in data.producers:
    data.LinInvCost[pp, 'solar'] = 4
    data.LinInvCost[pp, 'wind'] = 6.5
    #
    data.QuadInvCost[pp, 'solar'] = 0.004
    data.QuadInvCost[pp, 'wind'] = 0.016
    #
    data.RPS[pp] = 0.6
    
# Capacity
data.InitCapacity[('p1_1','coal')] = 250
data.InitCapacity[('p1_1','gas')] = 1275
data.InitCapacity[('p1_1','solar')] = 227.23
data.InitCapacity[('p1_1','wind')] = 428.65

data.InitCapacity[('p1_2','coal')] = 750
data.InitCapacity[('p1_2','gas')] = 425
data.InitCapacity[('p1_2','solar')] = 227.23
data.InitCapacity[('p1_2','wind')] = 428.65

data.InitCapacity[('p2_1','coal')] = 250
data.InitCapacity[('p2_1','gas')] = 1275
data.InitCapacity[('p2_1','solar')] = 227.23
data.InitCapacity[('p2_1','wind')] = 428.65

data.InitCapacity[('p2_2','coal')] = 750
data.InitCapacity[('p2_2','gas')] = 425
data.InitCapacity[('p2_2','solar')] = 227.23
data.InitCapacity[('p2_2','wind')] = 428.65



# Complementarity Problem Formulation

In [37]:
## Variables

M = Model()
# ## Variables
# ### Follower Variables 
PRODUCTION = M.addVars(data.producers, data.energy, data.scenario, vtype=GRB.CONTINUOUS, name = "PROD")
INVESTMENT = M.addVars(data.producers, data.energy, vtype=GRB.CONTINUOUS, name = "INV") 
FOLL_CARB_BUY = M.addVars(data.producers, lb = -GRB.INFINITY, name = "FOLL_CARB_BUY")

### All data.dirty investments must be 0 
for ee in data.dirty:
    for x in INVESTMENT.select("*",ee):
        x.ub = 0
        
# ### Leader Variables 
TOTAL_INV = M.addVars(data.countries, data.energy, name = "TOTAL_INV")
TOTAL_EMIT = M.addVars(data.countries, data.scenario, name = "TOTAL_EMIT")
C_PROD = M.addVars(data.countries, data.scenario, name="C_PROD")

# ### Duals 
D_INFRALIMIT = M.addVars(data.producers, data.energy, data.scenario,name = "D_INFRA")
D_EMITLIMIT = M.addVars(data.producers, data.scenario, name = "D_EMIT")
D_RPS = M.addVars(data.producers, data.scenario, name = "D_RPS")

## Constraints
for pp,ss in itertools.product(data.producers, data.scenario):
    eqn = quicksum([PRODUCTION[pp,gg,ss] for gg in data.green]) - PRODUCTION.sum(pp,'*',ss)*data.RPS[pp] 
    addComplementarity(M, D_RPS[pp,ss], eqn, name = "eq_rps"+"_".join([pp,ss]))


# Constraints
# ### Follower KKT conditions 

# eq_infraLimit
for pp,ee,ss in itertools.product(data.producers,data.energy,data.scenario):
    eqn = 0
    if ee in data.green:
        eqn = data.CapacityFactor[pp, ee, ss]*(INVESTMENT[pp,ee]+data.InitCapacity[pp, ee]) - PRODUCTION[pp, ee, ss]
    else:
        eqn = INVESTMENT[pp,ee]+data.InitCapacity[pp, ee]- PRODUCTION[pp, ee, ss]
    addComplementarity(M, D_INFRALIMIT[pp,ee,ss], eqn, name = "eq_infraLimit"+"_".join([pp,ee,ss]))

# eq_emitLimit
for pp,ss in itertools.product(data.producers, data.scenario):
    eqn = data.InitCredits[pp] + FOLL_CARB_BUY[pp] - quicksum(data.Emission[ee]*PRODUCTION[pp,ee,ss] for ee in data.dirty)
    addComplementarity(M, D_EMITLIMIT[pp,ss], eqn,name = "eq_emitLimit"+"_".join([pp,ss]))
    

# eq_countryProduction
for ss in data.scenario:
    for cc in data.countries:
        eqn = 0
        for pp in data.producers:
            if (cc,pp) in data.domesticity:
                eqn = eqn + quicksum(PRODUCTION[pp,ee,ss] for ee  in data.energy)
        M.addConstr(eqn == C_PROD[cc,ss], name="eq_countryProduction"+"_".join([cc,ss]))

# eq_production
for pp,ee,ss in itertools.product(data.producers,data.energy,data.scenario):
    eqn = data.probability[ss]*(
        data.LinProdCost[pp,ee] + data.QuadProdCost[pp,ee]*PRODUCTION[pp,ee,ss] -
        quicksum(data.DemInt[cc[0],ss] - data.DemSlope[cc[0],ss]*C_PROD[cc[0],ss] for cc in data.domesticity.select("*",pp)) +
        quicksum(data.DemSlope[cc[0],ss]*PRODUCTION[pp,ee2,ss] for cc in data.domesticity.select("*",pp) for ee2 in data.energy) 
    )+  D_INFRALIMIT[pp,ee,ss] + data.Emission[ee]*D_EMITLIMIT[pp,ss] + (data.RPS[pp] - (1 if ee in data.green else 0) )*D_RPS[pp,ss] 

    addComplementarity(M, PRODUCTION[pp,ee,ss],eqn,name = "eq_PRODUCTION"+"_".join([pp,ee,ss]))
                       
# eq_investment
for pp,ee in itertools.product(data.producers,data.green):
    eqn = data.LinInvCost[pp,ee] + data.QuadInvCost[pp,ee]*INVESTMENT[pp,ee] - quicksum(data.CapacityFactor[pp,ee,ss]*D_INFRALIMIT[pp,ee,ss] for ss in data.scenario)
    addComplementarity(M, INVESTMENT[pp,ee], eqn,name = "eq_INVESTMENT"+"_".join([pp,ee]))

# eq_carbonPurchase
for pp in data.producers:
    eqn = data.cciInt + data.cciSlope*FOLL_CARB_BUY.sum('*')  + data.cciSlope*FOLL_CARB_BUY[pp] -D_EMITLIMIT.sum(pp,'*')
    addComplementarity(M, FOLL_CARB_BUY[pp], eqn, name = "eq_carbonPurchase"+str(pp))

# Total data.Emission
for ss in data.scenario:
    eqn = {cc:0 for cc in data.countries}
    for (cc,pp) in data.domesticity:
        eqn[cc] = eqn[cc] + quicksum(data.Emission[ee]*PRODUCTION[pp,ee,ss] for ee in data.energy)
    M.addConstrs((TOTAL_EMIT[cc,ss] - eqn[cc]==0 for cc in data.countries), name="TotalEmission"+str(ss))

# Total data.Investment
for ee in data.green:
    eqn = {cc:0 for cc in data.countries}
    for (cc,pp) in data.domesticity:
        eqn[cc] = eqn[cc] + INVESTMENT[pp,ee] 
    M.addConstrs((TOTAL_INV[cc,ee] - eqn[cc]==0 for cc in data.countries), name="TotalInv"+"_".join([cc,ee]))

# M.Params.Heuristics=0
M.Params.AggFill=0
# M.Params.CutPasses=1
# M.Params.Presolve = 2
# M.Params.CutPasses = 1
# M.Params.PreQLinearize = 1
M.Params.Threads = 2
M.update()


Changed value of parameter AggFill to 0
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter Threads to 2
   Prev: 0  Min: 0  Max: 1024  Default: 0


In [38]:
M.setObjective(TOTAL_EMIT.sum("*"),sense=GRB.MINIMIZE)
M.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 60 rows, 124 columns and 240 nonzeros
Model fingerprint: 0x766edb76
Model has 104 general constraints
Variable types: 72 continuous, 52 integer (52 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 1e+03]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Presolve removed 15 rows and 53 columns
Presolve time: 0.09s
Presolved: 45 rows, 71 columns, 159 nonzeros
Presolved model has 26 SOS constraint(s)
Variable types: 42 continuous, 29 integer (29 binary)

Root relaxation: objective 7.282903e+02, 32 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  728.29026    0    7          -  728.29026      -     -    0s
     0     0  728.29026    0 

In [39]:
# printVars(INVESTMENT)
printVars(INVESTMENT)

 ('p1_1', 'wind') 	---	 866.0812117497434
 ('p1_1', 'solar') 	---	 2583.001688260772
 ('p1_2', 'wind') 	---	 857.8762168251737
 ('p1_2', 'solar') 	---	 2559.8956680708925
 ('p2_1', 'wind') 	---	 866.0812117497435
 ('p2_1', 'solar') 	---	 2583.001688260772
 ('p2_2', 'wind') 	---	 857.8762168251736
 ('p2_2', 'solar') 	---	 2559.8956680708916


In [40]:
printVars(FOLL_CARB_BUY)

 p1_1 	---	 325.4747605922729
 p1_2 	---	 402.815502147148
 p2_1 	---	 325.47476059227324
 p2_2 	---	 402.815502147148


In [261]:
for ee in data.energy:
    vv = PRODUCTION.sum("*",ee,"*").getValue()
    print ("Total", ee, "production:\t", vv, "---- per country ---", vv/2)

Total coal production:	 1791.8754888958877 ---- per country --- 895.9377444479438
Total gas production:	 2649.939090923702 ---- per country --- 1324.969545461851
Total wind production:	 1429.8448621183113 ---- per country --- 714.9224310591557
Total solar production:	 2015.6676410044797 ---- per country --- 1007.8338205022399


In [252]:
prodObj = {}
prodCost = {} # (pp,ss)
prod = {} #(pp, ee)
invCost = {}
carbCost = {}
energPrice = {}
revenue = {}

for pp in data.producers:
    for ss in data.scenario:
        prodCost[pp,ss] = 0
        prod[pp,ss] = 0
        for ee in data.energy:
            prodCost[pp,ss] += data.LinProdCost[pp,ee]*PRODUCTION[pp,ee,ss].X*data.probability[ss]
            prodCost[pp,ss] += data.QuadProdCost[pp,ee]*PRODUCTION[pp,ee,ss].X*PRODUCTION[pp,ee,ss].X*data.probability[ss]
            prod[pp,ss] += PRODUCTION[pp,ee,ss].X*data.probability[ss]
            
    invCost[pp] = 0
    for gg in data.green:
        invCost[pp] += data.LinInvCost[pp,gg]*INVESTMENT[pp,ee].X
        invCost[pp] += data.QuadInvCost[pp,gg]*INVESTMENT[pp,ee].X*INVESTMENT[pp,ee].X
    
    carbCost[pp] = (data.cciInt + data.cciSlope*sum([FOLL_CARB_BUY[ppp].X for ppp in data.producers]))*FOLL_CARB_BUY[pp].X

for cc,ss in itertools.product(data.countries,data.scenario):
    energPrice[cc,ss] = data.DemInt[cc,ss] - data.DemSlope[cc,ss] * C_PROD[cc,ss].X

for pp in data.producers:
    cntry = data.domesticity.select("*",pp)[0][0]
    revenue[pp] = 0
    for ss in data.scenario:
        revenue[pp] += energPrice[cc,ss]*prod[pp,ss]*data.probability[ss]
    
        
    prodObj[pp] = revenue[pp] + carbCost[pp] + invCost[pp] + sum(prodCost[pp,ss] for ss in data.scenario)

In [253]:
carbCost

{'p1_1': 6842.676433765622,
 'p1_2': 4823.187947262317,
 'p2_1': 6842.676433765627,
 'p2_2': 4823.18794726231}

In [254]:
printVars(FOLL_CARB_BUY)

 p1_1 	---	 324.724495547109
 p1_2 	---	 228.8881095378268
 p2_1 	---	 324.7244955471092
 p2_2 	---	 228.8881095378265


In [255]:
(data.cciInt + data.cciSlope*FOLL_CARB_BUY.sum("*")).getValue()

21.072252101698716